In [15]:
from sklearn import metrics
from scipy import stats
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
sns.set()
plt.rcParams["figure.figsize"] = (10,10)

In [42]:
participant_id_column = 'email. Email address'
data_path = "/home/kti01/Documents/My Files/Projects/Overlap/data/phase2/groups"
files = os.listdir(data_path)
participant_dict = {
    'participant': [],
    'position': [],
    'years_as_dermatologist': [],
    'experience_in_dermoscopy': [],
    'years_using_dermoscopy': [],
    'country': [],
}
for file in files:

    if not file.endswith('csv'):
        continue

    data = pd.read_csv(os.path.join(data_path, file)).dropna(subset='submitdate. Date submitted')
    for participant in data[participant_id_column]:
        
        df = data[data[participant_id_column] == participant].iloc[:, 81:].transpose().copy()
        participant_dict['participant'].append(participant)
        if df.iloc[0].item() != 'Other':
            participant_dict['position'].append(df.iloc[0].item())
        else:
            participant_dict['position'].append(df.iloc[1].item())
        participant_dict['years_as_dermatologist'].append(df.iloc[2].item())
        participant_dict['experience_in_dermoscopy'].append(df.iloc[4].item())
        participant_dict['years_using_dermoscopy'].append(df.iloc[6].item())
        participant_dict['country'].append(df.iloc[8].item())
      
    
participant_metadata = pd.DataFrame(participant_dict)

# Map participant Emails to Codes defined in participants.csv
participants_df = pd.read_csv("/home/kti01/Documents/My Files/Projects/Overlap/data/participants.csv")
participants_df['E-Mail'] = participants_df['E-Mail'].apply(lambda x: x.strip() if pd.notna(x) else x)
participants_code_dict = dict(zip(participants_df['E-Mail'], participants_df['Code']))

participant_metadata['participant'] = participant_metadata.apply(lambda row: participants_code_dict[row['participant']] 
                                         if row['participant'] in participants_code_dict and 
                                         pd.notna(participants_code_dict[row['participant']])
                                         else row['participant'], axis=1)


participant_id_column = 'email'
data_path = "/home/kti01/Documents/My Files/Projects/Overlap/data/phase3/groups"

files = os.listdir(data_path)
participant_dict = {
    'participant': [],
    'xai_preference': [],
    'feedback': []
}

for file in files:

    if not file.endswith('csv'):
        continue

    data = pd.read_csv(os.path.join(data_path, file)).dropna(subset='submitdate')
    for participant in data[participant_id_column]:
        
        df = data[data[participant_id_column] == participant].iloc[:, 81:].transpose().copy()
        participant_dict['participant'].append(participant)
        participant_dict['xai_preference'].append(df.iloc[0].item())
        participant_dict['feedback'].append(df.iloc[1].item())
        
participant_feedback = pd.DataFrame(participant_dict)
participant_metadata['xai_preference'] = participant_feedback['xai_preference'].replace({'Yes': 1, 'Ja': 1, 'No': 0, 'Nein': 0})
participant_metadata['feedback'] = participant_feedback['feedback']

In [43]:
years_as_dermatologist_mapping = {
    '5-10 Jahre': '5-10 Years',
    '> 10 Jahre': '> 10 Years',
    '3-5 Jahre': '3-5 Years',
    '1-3 Jahre': '1-3 Years'
}

experience_in_dermoscopy_mapping = {
    'Regelmäßige Nutzung und Weiterbildung': 'Regular use and education',
    'Regelmäßige Nutzung': 'Regular use',
    'Regelmäßige wissenschaftliche Auseinandersetzung (Papiere, Konferenzen, Lehre, u.ä.)': 'Regular scientific discussion (papers, conferences, teaching, etc.).',
    'Gelegentliche Nutzung': 'Occasional use',
    'Grundausbildung erhalten, aber seltene Nutzung': 'Basic education received, but rare use'
}

years_using_dermoscopy_mapping = {
    '> 10 Jahre': '> 10 Years',
    '5-10 Jahre': '5-10 Years',
    '3-5 Jahre': '3-5 Years',
    '1-3 Jahre': '1-3 Years',
    '< 1 Jahr': '< 1 Year'
}

participant_metadata['experience_in_dermoscopy'].replace(experience_in_dermoscopy_mapping, inplace=True)
participant_metadata['years_as_dermatologist'].replace(years_as_dermatologist_mapping, inplace=True) 
participant_metadata['years_using_dermoscopy'].replace(years_using_dermoscopy_mapping, inplace=True)

In [44]:
participant_metadata.to_csv("/home/kti01/Documents/My Files/Projects/Overlap/data/participant_metadata.csv", index=False)